<a href="https://colab.research.google.com/github/Armweak5104/Credit-fraud/blob/main/credit_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler


fraud_data = pd.read_csv("credit_card_fraud.csv")

X = fraud_data.drop(columns=['is_fraud','dob'])
y = fraud_data['is_fraud']

print(X.head())
print(y.head())

  trans_date_trans_time                   merchant       category     amt  \
0   2019-01-01 00:00:44  Heller, Gutmann and Zieme    grocery_pos  107.23   
1   2019-01-01 00:00:51             Lind-Buckridge  entertainment  220.11   
2   2019-01-01 00:07:27                  Kiehn Inc    grocery_pos   96.29   
3   2019-01-01 00:09:03                Beier-Hyatt   shopping_pos    7.77   
4   2019-01-01 00:21:32                 Bruen-Yost       misc_pos    6.85   

                       city state      lat      long  city_pop  \
0                    Orient    WA  48.8878 -118.2105       149   
1                Malad City    ID  42.1808 -112.2620      4154   
2                   Grenada    CA  41.6125 -122.5258       589   
3  High Rolls Mountain Park    NM  32.9396 -105.8189       899   
4                   Freedom    WY  43.0172 -111.0292       471   

                                 job                         trans_num  \
0  Special educational needs teacher  1f76529f8574734946361c461b02

In [79]:
#Cleaning non numerical data

X = pd.get_dummies(X, columns=['merchant', 'category', 'city', 'state', 'job'])

X['trans_year'] = pd.to_datetime(X['trans_date_trans_time']).dt.year
X['trans_month'] = pd.to_datetime(X['trans_date_trans_time']).dt.month
X['trans_day'] = pd.to_datetime(X['trans_date_trans_time']).dt.day
X['trans_int'] =  X['trans_num'].apply(lambda x: int(x, 16))

X.drop(columns=['trans_date_trans_time', 'trans_num'], inplace=True)

#Accounting for undersampling 
undersampler = RandomUnderSampler(sampling_strategy=0.3, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

In [80]:
#splitting data and standardizing

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [81]:
# Build the neural network
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(x_train.shape[1],)))

model.add(tf.keras.layers.Dense(units=64, activation='relu'))

model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [82]:
#Training and testing model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
152/152 [==============================] - 2s 5ms/step - loss: 0.4850 - accuracy: 0.7901 - val_loss: 0.4432 - val_accuracy: 0.8244
Epoch 2/10
152/152 [==============================] - 0s 3ms/step - loss: 0.2726 - accuracy: 0.8892 - val_loss: 0.4293 - val_accuracy: 0.8318
Epoch 3/10
152/152 [==============================] - 1s 4ms/step - loss: 0.1568 - accuracy: 0.9394 - val_loss: 0.4871 - val_accuracy: 0.8410
Epoch 4/10
152/152 [==============================] - 1s 3ms/step - loss: 0.0863 - accuracy: 0.9688 - val_loss: 0.5597 - val_accuracy: 0.8447
Epoch 5/10
152/152 [==============================] - 0s 3ms/step - loss: 0.0437 - accuracy: 0.9868 - val_loss: 0.6854 - val_accuracy: 0.8447
Epoch 6/10
152/152 [==============================] - 1s 3ms/step - loss: 0.0285 - accuracy: 0.9916 - val_loss: 0.7063 - val_accuracy: 0.8521
Epoch 7/10
152/152 [==============================] - 1s 3ms/step - loss: 0.0215 - accuracy: 0.9928 - val_loss: 0.7450 - val_accuracy: 0.8595
Epoch 